# Building Agents That Use Code

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

Alfred is planning a party at the Wayne family mansion and needs your help to ensure everything goes smoothly. To assist him, we'll apply what we've learned about how a multi-step `CodeAgent` operates.


## Let's login to our HF account to access the Inference API

In [1]:
import os
from huggingface_hub import login

login(new_session=False, token=os.getenv("HF_TOKEN"))

user = !huggingface-cli whoami
username = user[0]
print(f"Logged in as: {username}")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Logged in as: gperdrizet


## Selecting a Playlist for the Party Using `smolagents`

An important part of a successful party is the music. Alfred needs some help selecting the playlist. Luckily, `smolagents` has got us covered! We can build an agent capable of searching the web using DuckDuckGo. To give the agent access to this tool, we include it in the tool list when creating the agent.

For the model, we'll rely on `InferenceClientModel`, which provides access to Hugging Face's [Inference API](https://huggingface.co/docs/api-inference/index). The default model is `"Qwen/Qwen2.5-Coder-32B-Instruct"`, which is performant and available for fast inference, but you can select any compatible model from the Hub.

Running an agent is quite straightforward:

In [2]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=InferenceClientModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for the best music recommendations for a party at the Wayne's mansion.                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # First phase - Search for general party music recommendations                                                   
  party_music_recommendations = web_search(query="best music for a party")                                         
  print("Party Music Recommendations:\n", party_music_recommendations)                                             
                                                                                                                   
  # Second phase - Search for music that specifically fits Wayne's mansion theme                                   
  wayne_mansion_music = web_search(query="best music for Wayne's mansion party")                                   
  print("\nWayne's Mansion Music Recommendations:\n", wayne_mansion_music)                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Party Music Recommendations:
 ## Search Results

[The 75 Best Party Songs That Will Get Everyone Dancing - 
Gear4music](https://www.gear4music.com/blog/best-party-songs/)
Creating a good vibe at your event hinges on selecting the perfect soundtrack. So, let's look at the 75 best party 
songs for your any event.

[40 Best Party Songs | Songs To Dance To, Ranked By Our Editors - Time 
Out](https://www.timeout.com/music/best-party-songs)
The 40 best party songs ever made It's party time guys - don your dancing shoes and have your speaker at the ready

[Top 200 Most Requested Songs - DJ Event Planner](https://djeventplanner.com/mostrequested.htm)
Browse the most requested songs as compiled by DJ Event Planner users. Free to use and view.

[Best Songs To Party: DJ's Ultimate Party Songs Playlist](https://top40weekly.com/best-songs-to-party/)
Best Songs To Party: DJ's Ultimate Party Songs Playlist If you're looking for the ultimate playlist to get your 
party started, look no further than this best party songs playlist. Whether you're hosting a birthday bash, a 
backyard barbecue, or just a night in with friends, the right music can make all the difference.

[The 101 Best Party Songs Of All Time (Dance Songs) - MG - Music 
Grotto](https://www.musicgrotto.com/best-party-songs/)
Let's get up and start partying! This ultimate list of the best party and dance songs of all time is sure to get 
you up and moving.

[100 Best Party Songs: The Ultimate Party Playlist - Audio Captain](https://audiocaptain.com/best-party-songs/)
Best Party Songs: The Ultimate Party Playlist. Here's the 100 best party songs of all time you can use to create 
your ultimate party playlist in 2025.

[Best Party Songs That Always Get People Dancing 2025](https://yourmusicdepot.com/party-songs/)
Best Party Songs | Read up for the ultimate way to create a party playlist! Song lists added to help you find all 
the best party songs.

[50 Dance Party Songs to Get People on Their Feet - 
LoveToKnow](https://www.lovetoknow.com/celebrations/parties/dance-party-song)
The perfect dance party song will get the whole crowd going. Play any of these songs that will turn up the party 
and get people dancing.

[The 50 Best Music Playlists for Parties [2025] - Playlist 
Names](https://www.playlistnames.org/best-music-playlists-for-parties/)
Are you ready to turn your next gathering into an unforgettable bash? 🎊 The right music can elevate any party, 
transforming it from a simple get-together into a full-blown celebration! In this article, we've curated a list of 
50 best music playlists for parties that will keep your guests dancing all night long.

[45 Songs That Get Your Event Guests on the Dance Floor Every 
Time](https://hub.theeventplannerexpo.com/entertainment/35-songs-that-get-your-event-guests-on-the-dance-floor-ever
y-time)
While music genre preferences may differ, these are the biggest hits that motivate New York event guests to head to
the dance floor.

Wayne's Mansion Music Recommendations:
 ## Search Results

[The 75 Best Party Songs That Will Get Everyone Dancing - 
Gear4music](https://www.gear4music.com/blog/best-party-songs/)
Creating the perfect soundtrack is crucial to getting the right atmosphere at your event. If you're throwing a 
party, there are countless songs that will get everyone up and dancing; even once you've chosen your mix, you'll 
probably still think of more later! So, to keep things simple, we've compiled the best party songs of all time, 
from timeless classics to contemporary hits, giving ...

[Best Songs To Party: DJ's Ultimate Party Songs Playlist](https://top40weekly.com/best-songs-to-party/)
Best Songs To Party: DJ's Ultimate Party Songs Playlist If you're looking for the ultimate playlist to get your 
party started, look no further than this best party songs playlist. Whether you're hosting a birthday bash, a 
backyard barbecue, or just a night in with friends, the right music can make all the difference.

[The 50 Best Musi

[Step 1: Duration 6.71 seconds| Input tokens: 2,004 | Output tokens: 158]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Define potential songs from specific genres                                                                    
  potential_songs = [                                                                                              
      "The Final Countdown - Europe",    # Rock                                                                    
      "Sweet Child O' Mine - Guns N' Roses",  # Rock                                                               
      "Stairway to Heaven - Led Zeppelin",   # Rock/Blues                                                          
      "Bohemian Rhapsody - Queen",       # Rock/Blues                                                              
      "Creep - Radiohead",              # Gothic/Grunge                                                            
      "Smells Like Teen Spirit - Nirvana",   # Grunge                                                              
      "Under the Bridge - Red Hot Chili Peppers",  # Rock/Alternative                                              
      "Battery - Metallica",            # Hard Rock/Metal                                                          
      "Pump It - The Black Eyed Peas",     # Party/Pop                                                             
      "The Heart Wants What It Wants - Gwen Stefani",  # Pop/Rock                                                  
      "Sweet Symphony - Florence + The Machine", # Gothic/Alternative                                              
      "Bitter Sweet Symphony - The Verve", # Gothic/Alternative                                                    
      "Sad Machine - A Perfect Circle",  # Gothic/Industrial                                                       
      "The World Is Ending Tonight - Iron Maiden", # Heavy Metal                                                   
      "All Night Long (All Night) - Lionel Richie & The Commodores", # Dance                                       
      "Livin' on a Prayer - Bon Jovi",   # Rock                                                                    
      "Don't Stop Believin' - Journey",  # Rock/Survival                                                           
      "The Man Who Sold the World - David Bowie", # Glam Rock/Gothic                                               
      "Hotel California - Eagles",      # Rock                                                                     
      "Master of Puppets - Metallica",    # Heavy Metal                                                            
      "Where the Streets Have No Name - U2", # Rock/Gothic                                                         
      "Seven Nation Army - The White Stripes",  # Rock/Alternative                                                 
      "I Wanna Die Young - Mötley Crüe", # Rock                                                                    
      "Thriller - Michael Jackson",     # Dance                                                                    
      "Billie Jean - Michael Jackson",    # Dance                                                                  
      "Smack My Bitch Up - The Sugababes",  # Pop-Dance                                                            
      "Drowning - Panic! At The Disco",   # Pop-Rock                                                               
      "Shine - Tame Impala",             # Alternative                                                             
      "One More Time - Daft Punk",  # Dance/Electronic                                                             
      "Get Lucky - Daft Punk ft. Pharrell Williams", # Dance/Electronic                                            
      "Shape of You - Ed Sheeran",         # Pop/Dance                                                             
      "Thinking Out Loud - Ed Sheeran",      # Pop/Romant

Execution logs:

Filtered Songs for Wayne's Mansion Party:
['The Final Countdown - Europe', "Sweet Child O' Mine - Guns N' Roses", 'Stairway to Heaven - Led Zeppelin', 
'Bohemian Rhapsody - Queen', 'Creep - Radiohead', 'Smells Like Teen Spirit - Nirvana', 'Under the Bridge - Red Hot 
Chili Peppers', 'Battery - Metallica', 'Pump It - The Black Eyed Peas', 'The Heart Wants What It Wants - Gwen 
Stefani', 'Sweet Symphony - Florence + The Machine', 'Bitter Sweet Symphony - The Verve', 'Sad Machine - A Perfect 
Circle', 'The World Is Ending Tonight - Iron Maiden', 'All Night Long (All Night) - Lionel Richie & The 
Commodores', "Livin' on a Prayer - Bon Jovi", "Don't Stop Believin' - Journey", 'The Man Who Sold the World - David
Bowie', 'Hotel California - Eagles', 'Master of Puppets - Metallica', 'Where the Streets Have No Name - U2', 'Seven
Nation Army - The White Stripes', 'I Wanna Die Young - Mötley Crüe', 'Thriller - Michael Jackson', 'Billie Jean - 
Michael Jackson', 'Smack My Bitch Up - The Sugababes', 'Drowning - Panic! At The Disco', 'Shine - Tame Impala', 
'One More Time - Daft Punk', 'Get Lucky - Daft Punk ft. Pharrell Williams', 'Shape of You - Ed Sheeran', 'Thinking 
Out Loud - Ed Sheeran', 'Havana - Camila Cabello ft. Young Thug', 'Uptown Funk - Mark Ronson ft. Bruno Mars', 'Roar
- Katy Perry', 'Bangarang - Skrillex ft. Sirah', 'Stronger - Kanye West', 'Take My Breath Away - Berlin', 'I Just 
Called to Say I Love You - Stevie Wonder', 'Sweet Dreams (Are Made of This) - Eurythmics', 'Like a Prayer - 
Madonna', 'The Best is Yet to Come - Queen', 'My Immortal - Evanescence', 'Every Little Thing She Does Is Magic - 
The Police', 'The Sign - Ace of Base', 'Can You Feel the Love Tonight - Elton John', 'Wish upon a star - Disney', 
'Hero - Mariah Carey', "I Don't Want to Miss a Thing - Aerosmith"]

Out - Final answer: The Final Countdown - Europe
Sweet Child O' Mine - Guns N' Roses
Stairway to Heaven - Led Zeppelin
Bohemian Rhapsody - Queen
Creep - Radiohead
Smells Like Teen Spirit - Nirvana
Under the Bridge - Red Hot Chili Peppers
Battery - Metallica
Pump It - The Black Eyed Peas
The Heart Wants What It Wants - Gwen Stefani
Sweet Symphony - Florence + The Machine
Bitter Sweet Symphony - The Verve
Sad Machine - A Perfect Circle
The World Is Ending Tonight - Iron Maiden
All Night Long (All Night) - Lionel Richie & The Commodores
Livin' on a Prayer - Bon Jovi
Don't Stop Believin' - Journey
The Man Who Sold the World - David Bowie
Hotel California - Eagles
Master of Puppets - Metallica
Where the Streets Have No Name - U2
Seven Nation Army - The White Stripes
I Wanna Die Young - Mötley Crüe
Thriller - Michael Jackson
Billie Jean - Michael Jackson
Smack My Bitch Up - The Sugababes
Drowning - Panic! At The Disco
Shine - Tame Impala
One More Time - Daft Punk
Get Lucky - Daft Punk ft. Pharrell Williams
Shape of You - Ed Sheeran
Thinking Out Loud - Ed Sheeran
Havana - Camila Cabello ft. Young Thug
Uptown Funk - Mark Ronson ft. Bruno Mars
Roar - Katy Perry
Bangarang - Skrillex ft. Sirah
Stronger - Kanye West
Take My Breath Away - Berlin
I Just Called to Say I Love You - Stevie Wonder
Sweet Dreams (Are Made of This) - Eurythmics
Like a Prayer - Madonna
The Best is Yet to Come - Queen
My Immortal - Evanescence
Every Little Thing She Does Is Magic - The Police
The Sign - Ace of Base
Can You Feel the Love Tonight - Elton John
Wish upon a star - Disney
Hero - Mariah Carey
I Don't Want to Miss a Thing - Aerosmith

[Step 5: Duration 13.36 seconds| Input tokens: 24,182 | Output tokens: 2,038]

"The Final Countdown - Europe\nSweet Child O' Mine - Guns N' Roses\nStairway to Heaven - Led Zeppelin\nBohemian Rhapsody - Queen\nCreep - Radiohead\nSmells Like Teen Spirit - Nirvana\nUnder the Bridge - Red Hot Chili Peppers\nBattery - Metallica\nPump It - The Black Eyed Peas\nThe Heart Wants What It Wants - Gwen Stefani\nSweet Symphony - Florence + The Machine\nBitter Sweet Symphony - The Verve\nSad Machine - A Perfect Circle\nThe World Is Ending Tonight - Iron Maiden\nAll Night Long (All Night) - Lionel Richie & The Commodores\nLivin' on a Prayer - Bon Jovi\nDon't Stop Believin' - Journey\nThe Man Who Sold the World - David Bowie\nHotel California - Eagles\nMaster of Puppets - Metallica\nWhere the Streets Have No Name - U2\nSeven Nation Army - The White Stripes\nI Wanna Die Young - Mötley Crüe\nThriller - Michael Jackson\nBillie Jean - Michael Jackson\nSmack My Bitch Up - The Sugababes\nDrowning - Panic! At The Disco\nShine - Tame Impala\nOne More Time - Daft Punk\nGet Lucky - Daft P

When you run this example, the output will **display a trace of the workflow steps being executed**. It will also print the corresponding Python code with the message:

```python
 ─ Executing parsed code: ────────────────────────────────────────────────────────────────────────────────────────
  results = web_search(query="best music for a Batman party")                                                      
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
```

After a few steps, you'll see the generated playlist that Alfred can use for the party! 🎵

## Using a Custom Tool to Prepare the Menu

Now that we have selected a playlist, we need to organize the menu for the guests. Again, Alfred can take advantage of `smolagents` to do so. Here, we use the `@tool` decorator to define a custom function that acts as a tool. We'll cover tool creation in more detail later, so for now, we can simply run the code.

As you can see in the example below, we will create a tool using `@tool` decorator and include it in the `tools` list.

In [3]:
from smolagents import CodeAgent, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

agent = CodeAgent(tools=[suggest_menu], model=InferenceClientModel())

agent.run("Prepare a formal menu for the party.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Prepare a formal menu for the party.                                                                            │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  formal_menu = suggest_menu(occasion="formal")                                                                    
  print(formal_menu)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
3-course dinner with wine and dessert.

Out: None

[Step 1: Duration 0.89 seconds| Input tokens: 2,026 | Output tokens: 51]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  formal_menu = (                                                                                                  
      "Appetizer: Smoked Salmon Tartare with Crème Fraîche and Dill\n"                                             
      "Main Course: Rack of Lamb with Roasted Vegetables and Herb Crust\n"                                         
      "Wine Pairing: Aged Pinot Noir\n"                                                                            
      "Dessert: Chocolate Fondant with Vanilla Ice Cream"                                                          
  )                                                                                                                
  final_answer(formal_menu)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Appetizer: Smoked Salmon Tartare with Crème Fraîche and Dill
Main Course: Rack of Lamb with Roasted Vegetables and Herb Crust
Wine Pairing: Aged Pinot Noir
Dessert: Chocolate Fondant with Vanilla Ice Cream

[Step 3: Duration 2.16 seconds| Input tokens: 6,511 | Output tokens: 257]

'Appetizer: Smoked Salmon Tartare with Crème Fraîche and Dill\nMain Course: Rack of Lamb with Roasted Vegetables and Herb Crust\nWine Pairing: Aged Pinot Noir\nDessert: Chocolate Fondant with Vanilla Ice Cream'

The agent will run for a few steps until finding the answer.

The menu is ready! 🥗

## Using Python Imports Inside the Agent

We have the playlist and menu ready, but we need to check one more crucial detail: preparation time!

Alfred needs to calculate when everything would be ready if he started preparing now, in case they need assistance from other superheroes.

`smolagents` specializes in agents that write and execute Python code snippets, offering sandboxed execution for security. It supports both open-source and proprietary language models, making it adaptable to various development environments.

**Code execution has strict security measures** - imports outside a predefined safe list are blocked by default. However, you can authorize additional imports by passing them as strings in `additional_authorized_imports`.
For more details on secure code execution, see the official [guide](https://huggingface.co/docs/smolagents/tutorials/secure_code_execution).

When creating the agent, we ill use `additional_authorized_imports` to allow for importing the `datetime` module.

In [4]:
from smolagents import CodeAgent, InferenceClientModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[], model=InferenceClientModel(), additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    3. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Alfred needs to prepare for the party. Here are the tasks:                                                      │
│     1. Prepare the drinks - 30 minutes                                                                          │
│     2. Decorate the mansion - 60 minutes                                                                        │
│     3. Set up the menu - 45 minutes                                                                             │
│     3. Prepare the music and playlist - 45 minutes                                                              │
│                                                                                                                 │
│     If we start right now, at what time will the party be ready?                                                │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import datetime                                                                                                  
                                                                                                                   
  # Define task durations in minutes                                                                               
  task_durations = {                                                                                               
      "Prepare the drinks": 30,                                                                                    
      "Decorate the mansion": 60,                                                                                  
      "Set up the menu": 45,                                                                                       
      "Prepare the music and playlist": 45                                                                         
  }                                                                                                                
                                                                                                                   
  # Calculate total time required                                                                                  
  total_time_minutes = sum(task_durations.values())                                                                
                                                                                                                   
  # Convert total time to timedelta                                                                                
  total_time_timedelta = datetime.timedelta(minutes=total_time_minutes)                                            
                                                                                                                   
  # Get current time                                                                                               
  current_time = datetime.datetime.now()                                                                           
                                                                                                                   
  # Calculate the time when the party will be ready                                                                
  party_ready_time = current_time + total_time_timedelta                                                           
                                                                                                                   
  # Print the total time and the party ready time                                                                  
  print("Total time required (minutes):", total_time_minutes)                                                      
  print("Party will be ready at:", party_ready_time.strftime("%Y-%m-%d %H:%M:%S"))                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Total time required (minutes): 180
Party will be ready at: 2025-06-28 01:58:39

Out: None

[Step 1: Duration 3.29 seconds| Input tokens: 2,025 | Output tokens: 237]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("2025-06-28 01:58:39")                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 2025-06-28 01:58:39

[Step 2: Duration 1.82 seconds| Input tokens: 4,582 | Output tokens: 345]

'2025-06-28 01:58:39'

These examples are just the beginning of what you can do with code agents, and we're already starting to see their utility for preparing the party.
You can learn more about how to build code agents in the [smolagents documentation](https://huggingface.co/docs/smolagents).

`smolagents` specializes in agents that write and execute Python code snippets, offering sandboxed execution for security. It supports both open-source and proprietary language models, making it adaptable to various development environments.

## Sharing Our Custom Party Preparator Agent to the Hub

Wouldn't it be **amazing to share our very own Alfred agent with the community**? By doing so, anyone can easily download and use the agent directly from the Hub, bringing the ultimate party planner of Gotham to their fingertips! Let's make it happen! 🎉

The `smolagents` library makes this possible by allowing you to share a complete agent with the community and download others for immediate use. It's as simple as the following:


In [5]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel, VisitWebpageTool, FinalAnswerTool, Tool, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
        ],
    model=InferenceClientModel(),
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To come up with the best playlist for a 'villain masquerade' themed party at Wayne's mansion, I will start
by generating some superhero party themes related to villains. Then, using these themes, I will suggest a menu and 
perform a web search to find appropriate music playlists.                                                          
                                                                                                                   
<code>                                                                                                             
theme = superhero_party_theme_generator(category="villain masquerade")                                             
print(f"Generated Party Theme: {theme}")                                                                           
</code>                                                                                                            

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  theme = superhero_party_theme_generator(category="villain masquerade")                                           
  print(f"Generated Party Theme: {theme}")                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Generated Party Theme: Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Since there is a rate limit issue with the web search, I'll manually select some tracks from the          
previously identified Spotify playlist that have a villainous or mysterious atmosphere. Then I'll also add some    
classical pieces from the Gotham soundtracks that are likely to fit the theme, choosing them based on my knowledge 
of these soundtracks rather than relying on additional web searches.                                               
                                                                                                                   
Selected tracks from Spotify playlist:                                                                             
- Overture - From 'The Phantom Of The Opera' Motion Picture                                                        
- Serenade for Strings in E Major, Op. 22, B. 52: II. Tempo di valse                                               
- Devil's Backbone                                                                                                 
- Romeo and Juliet, Op. 64 / Act 1: Dance Of The Knights                                                           
- Skyfall                                                                                                          
- The Second Waltz                                                                                                 
- Tchaikovsky: Swan Lake, Op. 20, Act 2: No. 10, Scene. Moderato                                                   
                                                                                                                   
Selected classical tracks from Gotham soundtracks:                                                                 
- Joker's Theme - Opening Theme (Season 1)                                                                         
- Bane's Theme - Main Title (Season 1)                                                                             
- Selina Kyle's Theme - Catwoman Main Theme (Season 1)                                                             
- Two-Face's Theme - Harvey Dent/Bruce Wayne (Season 2)                                                            
- Mr. Freeze's Theme - Victor Fries Main Theme (Season 2)                                                          
- Harley Quinn's Theme

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  tracks_spotify = [                                                                                               
      "Overture - From 'The Phantom Of The Opera' Motion Picture",                                                 
      "Serenade for Strings in E Major, Op. 22, B. 52: II. Tempo di valse",                                        
      "Devil's Backbone",                                                                                          
      "Romeo and Juliet, Op. 64 / Act 1: Dance Of The Knights",                                                    
      "Skyfall",                                                                                                   
      "The Second Waltz",                                                                                          
      "Tchaikovsky: Swan Lake, Op. 20, Act 2: No. 10, Scene. Moderato"                                             
  ]                                                                                                                
                                                                                                                   
  tracks_gotham = [                                                                                                
      "Joker's Theme - Opening Theme (Season 1)",                                                                  
      "Bane's Theme - Main Title (Season 1)",                                                                      
      "Selina Kyle's Theme - Catwoman Main Theme (Season 1)",                                                      
      "Two-Face's Theme - Harvey Dent/Bruce Wayne (Season 2)",                                                     
      "Mr. Freeze's Theme - Victor Fries Main Theme (Season 2)",                                                   
      "Harley Quinn's Theme - Criminal Instinct (Season 2)"                                                        
  ]                                                                                                                
                                                                                                                   
  best_playlist = tracks_spotify + tracks_gotham                                                                   
  final_answer(best_playlist)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: ["Overture - From 'The Phantom Of The Opera' Motion Picture", 'Serenade for Strings in E Major,
Op. 22, B. 52: II. Tempo di valse', "Devil's Backbone", 'Romeo and Juliet, Op. 64 / Act 1: Dance Of The Knights', 
'Skyfall', 'The Second Waltz', 'Tchaikovsky: Swan Lake, Op. 20, Act 2: No. 10, Scene. Moderato', "Joker's Theme - 
Opening Theme (Season 1)", "Bane's Theme - Main Title (Season 1)", "Selina Kyle's Theme - Catwoman Main Theme 
(Season 1)", "Two-Face's Theme - Harvey Dent/Bruce Wayne (Season 2)", "Mr. Freeze's Theme - Victor Fries Main Theme
(Season 2)", "Harley Quinn's Theme - Criminal Instinct (Season 2)"]

[Step 6: Duration 7.02 seconds| Input tokens: 27,216 | Output tokens: 1,188]

["Overture - From 'The Phantom Of The Opera' Motion Picture",
 'Serenade for Strings in E Major, Op. 22, B. 52: II. Tempo di valse',
 "Devil's Backbone",
 'Romeo and Juliet, Op. 64 / Act 1: Dance Of The Knights',
 'Skyfall',
 'The Second Waltz',
 'Tchaikovsky: Swan Lake, Op. 20, Act 2: No. 10, Scene. Moderato',
 "Joker's Theme - Opening Theme (Season 1)",
 "Bane's Theme - Main Title (Season 1)",
 "Selina Kyle's Theme - Catwoman Main Theme (Season 1)",
 "Two-Face's Theme - Harvey Dent/Bruce Wayne (Season 2)",
 "Mr. Freeze's Theme - Victor Fries Main Theme (Season 2)",
 "Harley Quinn's Theme - Criminal Instinct (Season 2)"]

In [6]:
agent.push_to_hub(f'{username}/AlfredAgent')

README.md:   0%|          | 0.00/267 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


This agent has step_callbacks: they will be ignored by this method. LogLevel.INFO

CommitInfo(commit_url='https://huggingface.co/spaces/gperdrizet/AlfredAgent/commit/b4eee455ec30a5c81196fbe4c7e0b027672b017f', commit_message='Upload agent', commit_description='', oid='b4eee455ec30a5c81196fbe4c7e0b027672b017f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/gperdrizet/AlfredAgent', endpoint='https://huggingface.co', repo_type='space', repo_id='gperdrizet/AlfredAgent'), pr_revision=None, pr_num=None)

To download the agent again, use the code below:

In [7]:
agent = CodeAgent(tools=[], model=InferenceClientModel())
alfred_agent = agent.from_hub(f'{username}/AlfredAgent', trust_remote_code=True)

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

prompts.yaml: 0.00B [00:00, ?B/s]

agent.json: 0.00B [00:00, ?B/s]

visit_webpage.py: 0.00B [00:00, ?B/s]

In [8]:
alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: To come up with the best playlist for a party at Wayne's mansion with a 'villain masquerade' theme, I     
should first use the `superhero_party_theme_generator` tool to get some specific theme ideas related to villains.  
Then, I will use the `web_search` tool to find playlists that match these themes.                                  
                                                                                                                   
<code>                                                                                                             
villain_theme_ideas = superhero_party_theme_generator(category="villain masquerade")                               
print(villain_theme_ideas)                                                                                         
</code>                                                                                                            

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  villain_theme_ideas = superhero_party_theme_generator(category="villain masquerade")                             
  print(villain_theme_ideas)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.

Out: None

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Now that I have compiled a list of themed songs, I can present this playlist as the best option for a     
party at Wayne's mansion with a 'villain masquerade' theme.                                                        
                                                                                                                   
<code>                                                                                                             
final_playlist = """                                                                                               
**Wayne's Mansion Villain Masquerade Playlist:**                                                                   
                                                                                                                   
1. Villain - Stella Jang                                                                                           
2. Therefore I Am - Billie Eilish                                                                                  
3. Posing In Bondage - Japanese Breakfast                                                                          
4. Savage Good Boy - Japanese Breakfast                                                                            
5. Like in the Movies - Vlad Holiday                                                                               
6. Happy Pill - grandson, Mob Rich                                                                                 
7. Yoko Ono - Mob Rich                                                                                             
8. Sweet Talk - Saint Motel                                                                                        
9. Holy Ghost - BØRNS                                                                                              
10. Past Lives - BØRNS                                                                                             
11. Church (feat. nothing, nowhere.) remix - Fall Out Boy, nothing,nowhere., jay vee                               
12. Gooey - Stripped - Glass Animals                                                                               
13. Black Mambo - Glass Animals                                                                                    
"""                                                                                                                
                                                                                                                   
final_answer(final_playlist)                                                                                       
</code>                                                                                                            
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_playlist = """                                                                                             
  **Wayne's Mansion Villain Masquerade Playlist:**                                                                 
                                                                                                                   
  1. Villain - Stella Jang                                                                                         
  2. Therefore I Am - Billie Eilish                                                                                
  3. Posing In Bondage - Japanese Breakfast                                                                        
  4. Savage Good Boy - Japanese Breakfast                                                                          
  5. Like in the Movies - Vlad Holiday                                                                             
  6. Happy Pill - grandson, Mob Rich                                                                               
  7. Yoko Ono - Mob Rich                                                                                           
  8. Sweet Talk - Saint Motel                                                                                      
  9. Holy Ghost - BØRNS                                                                                            
  10. Past Lives - BØRNS                                                                                           
  11. Church (feat. nothing, nowhere.) remix - Fall Out Boy, nothing,nowhere., jay vee                             
  12. Gooey - Stripped - Glass Animals                                                                             
  13. Black Mambo - Glass Animals                                                                                  
  """                                                                                                              
                                                                                                                   
  final_answer(final_playlist)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
**Wayne's Mansion Villain Masquerade Playlist:**

1. Villain - Stella Jang
2. Therefore I Am - Billie Eilish
3. Posing In Bondage - Japanese Breakfast
4. Savage Good Boy - Japanese Breakfast
5. Like in the Movies - Vlad Holiday
6. Happy Pill - grandson, Mob Rich
7. Yoko Ono - Mob Rich
8. Sweet Talk - Saint Motel
9. Holy Ghost - BØRNS
10. Past Lives - BØRNS
11. Church (feat. nothing, nowhere.) remix - Fall Out Boy, nothing,nowhere., jay vee
12. Gooey - Stripped - Glass Animals
13. Black Mambo - Glass Animals

[Step 6: Duration 3.37 seconds| Input tokens: 25,336 | Output tokens: 1,095]

"\n**Wayne's Mansion Villain Masquerade Playlist:**\n\n1. Villain - Stella Jang\n2. Therefore I Am - Billie Eilish\n3. Posing In Bondage - Japanese Breakfast\n4. Savage Good Boy - Japanese Breakfast\n5. Like in the Movies - Vlad Holiday\n6. Happy Pill - grandson, Mob Rich\n7. Yoko Ono - Mob Rich\n8. Sweet Talk - Saint Motel\n9. Holy Ghost - BØRNS\n10. Past Lives - BØRNS\n11. Church (feat. nothing, nowhere.) remix - Fall Out Boy, nothing,nowhere., jay vee\n12. Gooey - Stripped - Glass Animals\n13. Black Mambo - Glass Animals\n"

What's also exciting is that shared agents are directly available as Hugging Face Spaces, allowing you to interact with them in real-time. You can explore other agents [here](https://huggingface.co/spaces/davidberenstein1957/smolagents-and-tools).

For example, the _AlfredAgent_ is available [here](https://huggingface.co/spaces/sergiopaniego/AlfredAgent).

### Inspecting Our Party Preparator Agent with OpenTelemetry and Langfuse 📡

Full trace can be found [here](https://cloud.langfuse.com/project/cm7bq0abj025rad078ak3luwi/traces/995fc019255528e4f48cf6770b0ce27b?timestamp=2025-02-19T10%3A28%3A36.929Z).

As Alfred fine-tunes the Party Preparator Agent, he's growing weary of debugging its runs. Agents, by nature, are unpredictable and difficult to inspect. But since he aims to build the ultimate Party Preparator Agent and deploy it in production, he needs robust traceability for future monitoring and analysis.  

Once again, `smolagents` comes to the rescue! It embraces the [OpenTelemetry](https://opentelemetry.io/) standard for instrumenting agent runs, allowing seamless inspection and logging. With the help of [Langfuse](https://langfuse.com/) and the `SmolagentsInstrumentor`, Alfred can easily track and analyze his agent’s behavior.  

Alfred has already created an account on Langfuse and has his API keys ready. If you haven’t done so yet, you can sign up for Langfuse Cloud [here](https://cloud.langfuse.com/) or explore [alternatives](https://huggingface.co/docs/smolagents/tutorials/inspect_runs).  

Once you have your API keys, they need to be properly configured as follows:

In [9]:
import os
import base64

LANGFUSE_PUBLIC_KEY=os.environ["LANGFUSE_PUBLIC_KEY"]
LANGFUSE_SECRET_KEY=os.environ["LANGFUSE_SECRET_KEY"]
LANGFUSE_AUTH=base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()

#os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel" # EU data region
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel" # US data region
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

Finally, Alfred is ready to initialize the `SmolagentsInstrumentor` and start tracking his agent's performance.  

In [10]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

Alfred is now connected 🔌! The runs from `smolagents` are being logged in Langfuse, giving him full visibility into the agent's behavior. With this setup, he's ready to revisit previous runs and refine his Party Preparator Agent even further.  

In [11]:
from smolagents import CodeAgent, InferenceClientModel

agent = CodeAgent(tools=[], model=InferenceClientModel())
alfred_agent = agent.from_hub(f'{username}/AlfredAgent', trust_remote_code=True)
alfred_agent.run("Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme        │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/mnt/arkk/hf-agents-course/unit-two-frameworks/.venv/lib/python3.10/site-packages/smolagents/models.py:393: FutureWarning: Attribute last_input_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.input_tokens instead.
  warnings.warn(
/mnt/arkk/hf-agents-course/unit-two-frameworks/.venv/lib/python3.10/site-packages/smolagents/models.py:402: FutureWarning: Attribute last_output_token_count is deprecated and will be removed in version 1.20. Please use TokenUsage.output_tokens instead.
  warnings.warn(


Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: I need to generate a menu based on the 'villain masquerade' theme and then suggest a best playlist for    
that theme.                                                                                                        
                                                                                                                   
<code>                                                                                                             
party_theme = superhero_party_theme_generator(category="villain masquerade")                                       
print(f"The party theme suggested is: {party_theme}")                                                              
</code>                                                                                                            

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Thought: Now that I have the list of songs for the villain masquerade theme, I will present this playlist as the   
final answer.                                                                                                      
                                                                                                                   
<code>                                                                                                             
final_playlist = "Best Playlist for Wayne's Mansion Villain Masquerade:\n" + "\n".join([f"* {song}" for song in    
villain_playlist])                                                                                                 
final_answer(final_playlist)                                                                                       
</code>                                                                                                            
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_playlist = "Best Playlist for Wayne's Mansion Villain Masquerade:\n" + "\n".join([f"* {song}" for song in  
  villain_playlist])                                                                                               
  final_answer(final_playlist)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Best Playlist for Wayne's Mansion Villain Masquerade:
* Bad Guy - Billie Eilish
* Smooth Criminal - Michael Jackson
* Heathens - Twenty One Pilots
* The Imperial March (Darth Vader’s Theme) - John Williams
* Disturbia - Rihanna
* Throne - Bring Me The Horizon
* Seven Nation Army - The White Stripes
* Paint It, Black - The Rolling Stones
* Toxic - Britney Spears
* Black Skinhead - Kanye West

[Step 5: Duration 2.09 seconds| Input tokens: 31,104 | Output tokens: 564]

"Best Playlist for Wayne's Mansion Villain Masquerade:\n* Bad Guy - Billie Eilish\n* Smooth Criminal - Michael Jackson\n* Heathens - Twenty One Pilots\n* The Imperial March (Darth Vader’s Theme) - John Williams\n* Disturbia - Rihanna\n* Throne - Bring Me The Horizon\n* Seven Nation Army - The White Stripes\n* Paint It, Black - The Rolling Stones\n* Toxic - Britney Spears\n* Black Skinhead - Kanye West"

Alfred can now access logs [here](https://cloud.langfuse.com) to review and analyze them.  

Meanwhile, the [suggested playlist](https://open.spotify.com/playlist/0gZMMHjuxMrrybQ7wTMTpw) sets the perfect vibe for the party preparations. Cool, right? 🎶
